In [678]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import emcee
import random

In [679]:
def const_fn_red_chi2(y_and_sd, const):
    chi2 = 0
    for tup in y_and_sd:
        y, sd = tup
        chi2 += ((y-const)/sd)**2
    df = len(y_and_sd) - 1
    return chi2/df

In [680]:
# file = fits.open('../data/uv_test/3c84.uvfits')

In [681]:
def naive_uv_fitting(data):#fits_file: str):
    # file = fits.open(fits_file)
    # data = file[1].data
    vis = np.array(data)
    freq_bin, u, v, Re, Im, w = [], [], [], [], [], []
    for row in vis:
        freq_bin_data, u_data, v_data, Re_data, Im_data, w_data = row
        freq_bin.append(int(freq_bin_data))
        u.append(int(u_data))
        v.append(int(v_data))
        Re.append(float(Re_data/w_data))
        Im.append(float(Im_data/w_data))
        w.append(float(w_data))

    # adding in conjugate half of data
    freq_bin *= 2
    neg_u = [-1 * val for val in u]
    u += neg_u
    neg_v = [-1 * val for val in v]
    v += neg_v
    Re *= 2
    neg_Im = [-1 * val for val in Im]
    Im += neg_Im
    w *= 2
    Rlambda = []
    for i in range(len(u)):
        Rlambda.append((u[i]**2 +v[i]**2)**(1/2))

    sd = [weight**(-1/2) for weight in w]
    amp_dat = []
    re_dat = []
    im_dat = []
    for i in range(len(sd)):
        amp_dat.append(((Re[i]**2 + Im[i]**2)**(1/2), sd[i]))
        re_dat.append((Re[i], sd[i]))
        im_dat.append((Im[i], sd[i]))
    zeroes = [0]*len(amp_dat)

    amp_popt, amp_pcov = curve_fit(const_fn_red_chi2, amp_dat, zeroes)
    amp_red_chi2 = const_fn_red_chi2(amp_dat, float(amp_popt))
    amp_dict = {'amplitude': float(amp_popt[0]), 'standard deviation': float(amp_pcov[0][0]**(1/2)), 'reduced chi2': amp_red_chi2}
    print(amp_popt, amp_pcov[0][0]**(1/2), amp_red_chi2)

    re_popt, re_pcov = curve_fit(const_fn_red_chi2, re_dat, zeroes)
    re_red_chi2 = const_fn_red_chi2(re_dat, float(re_popt))
    print(re_popt, re_pcov[0][0]**(1/2), re_red_chi2)

    im_popt, im_pcov = curve_fit(const_fn_red_chi2, im_dat, zeroes)
    im_red_chi2 = const_fn_red_chi2(im_dat, float(im_popt))
    print(im_popt, im_pcov[0][0]**(1/2), im_red_chi2)

In [682]:
# naive_uv_fitting('../data/uv_test/3c84.uvfits')

In [683]:
# logarithmic likelihood function (for Gaussian probability)
def log_likelihood(b, y, yerr):
    model = b
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model) ** 2 / sigma2 + np.log(2*np.pi*sigma2))

In [684]:
def log_prior(b):
    if 0.0 < b < 20.0: # this range is for testing purposes
        return 0.0
    return -np.inf

In [685]:
def log_probability(b, y, yerr):
    lp = log_prior(b)
    if not np.isfinite(lp): # if outside of range where m, b, log(f) could be
        return -np.inf # essentially 0 probability (e^-inf = 0)
    return lp + log_likelihood(b, y, yerr)

In [790]:
def mcmc_single_pt_source(data):#fits_file: str):
    # TODO: generalize to fitting n point/disk/Gaussian sources

    # data = fits_file[1].data
    vis = np.array(data)
    freq_bin, u, v, Re, Im, w = [], [], [], [], [], []
    for row in vis:
        freq_bin_data, u_data, v_data, Re_data, Im_data, w_data = row
        freq_bin.append(int(freq_bin_data))
        u.append(int(u_data))
        v.append(int(v_data))
        Re.append(float(Re_data/w_data))
        Im.append(float(Im_data/w_data))
        w.append(float(w_data))

    # adding in conjugate half of data
    freq_bin *= 2
    neg_u = [-1 * val for val in u]
    u += neg_u
    neg_v = [-1 * val for val in v]
    v += neg_v
    Re *= 2
    neg_Im = [-1 * val for val in Im]
    Im += neg_Im
    w *= 2
    Rlambda = []
    for i in range(len(u)):
        Rlambda.append((u[i]**2 +v[i]**2)**(1/2))

    amp = []
    for i in range(len(Re)):
        amp.append((Re[i]**2 + Im[i]**2)**(1/2))
    sd = [weight**(-1/2) for weight in w]

    # TODO: optional parameters: initial position, nwalkers, number of sources (therefore number of dimensions), nsteps, discard number
    pos = np.random.randn(50,1) * 2 + 10
    nwalkers = 50
    ndim = 1
    sampler = emcee.EnsembleSampler(
        nwalkers, ndim, log_probability, args=(np.array(amp), np.array(sd))
    )
    results = sampler.run_mcmc(pos, 5000)
    return results

In [791]:
# pos = np.random.randn(50,1) * 10 + 10
# nwalkers = 50
# ndim = 1
# sampler = emcee.EnsembleSampler(
#     nwalkers, ndim, log_probability, args=(np.array(amp), np.array(sd))
# )
# sampler.run_mcmc(pos, 5000)

In [792]:
# tau = sampler.get_autocorr_time()
# print(tau)

In [793]:
# # extract visibility data
# file = fits.open('../data/uv_test/3c84.uvfits')
# data = file[1].data
# vis = np.array(data)
# freq_bin, u, v, Re, Im, w = [], [], [], [], [], []
# for row in vis:
#     freq_bin_data, u_data, v_data, Re_data, Im_data, w_data = row
#     freq_bin.append(int(freq_bin_data))
#     u.append(int(u_data))
#     v.append(int(v_data))
#     Re.append(float(Re_data/w_data))
#     Im.append(float(Im_data/w_data))
#     w.append(float(w_data))

# # adding in conjugate half of data
# freq_bin *= 2
# neg_u = [-1 * val for val in u]
# u += neg_u
# neg_v = [-1 * val for val in v]
# v += neg_v
# Re *= 2
# neg_Im = [-1 * val for val in Im]
# Im += neg_Im
# w *= 2
# Rlambda = []
# for i in range(len(u)):
#     Rlambda.append((u[i]**2 +v[i]**2)**(1/2))

In [794]:
# # plot visibility data
# fig=plt.figure(figsize=(15,8))
# ax=fig.add_subplot(111)
# meters_u = [u_val / 1300 for u_val in u]
# meters_v = [v_val / 1300 for v_val in v]
# ax.scatter(meters_u,meters_v, c=Re, marker='.', cmap='gist_rainbow')
# ax.set_aspect(aspect=1.0)
# ax.set_xlabel('u (lambda)')
# ax.set_ylabel('v (lambda)')

In [795]:
# plt.scatter(Rlambda, Re, marker='.')
# weights = np.array(w)
# sd = weights**(-1/2)
# plt.errorbar(x=Rlambda, y=Re, yerr=sd, linestyle='None')

In [796]:
# np.ma.average(Re, axis=(0,40000,80000,100000,120000,140000,160000,180000))

In [797]:
# plt.scatter(Rlambda, Im, marker='.')
# plt.errorbar(x=Rlambda, y=Im, yerr=sd, linestyle='None')

In [798]:
# sd = [weight**(-1/2) for weight in w]

# uv_dat = []
# for i in range(len(sd)):
#     uv_dat.append(((Re[i]**2 + Im[i]**2)**(-1/2), sd[i]))
# zeroes = [0]*len(uv_dat)
# popt, pcov = curve_fit(const_fn_red_chi2, uv_dat, zeroes)
# red_chi2_val = const_fn_red_chi2(uv_dat, float(popt))

In [799]:
# red_chi2_val

In [800]:
##### FAKE ARRAY IN FORM OF KARTO'S DATA #####
fake_arr = np.tile([float(0)]*6, (20, 1))
for i in range(len(fake_arr)):
    fake_arr[i][1] = i
    fake_arr[i][2] = i
    real = 10
    imag = 0
    x = 1e-3
    re_noise = random.random() * x
    im_noise = random.random() * x
    if random.randint(0,1):
        re_noise *= -1
    if random.randint(0,1):
        im_noise *= -1
    fake_arr[i][3] = real + re_noise
    fake_arr[i][4] = imag + im_noise
    weight = (re_noise**2 + im_noise**2)**(-1)
    fake_arr[i][5] = weight
    fake_arr[i][3] *= weight # weighting reals like Karto
    fake_arr[i][4] *= weight # weighting imaginaries like Karto

In [801]:
naive_uv_fitting(fake_arr)

[10.00000947] 0.1972527850655273 0.4964300184009456
[10.00000947] 0.1972089024224183 0.4964279158315618
[8.2570801e-09] 0.04691238455294544 0.5287050121528215


/var/folders/3j/z5bj43t10vsdnks2s1xw3vm80000gn/T/ipykernel_94532/2823456958.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  amp_red_chi2 = const_fn_red_chi2(amp_dat, float(amp_popt))
/var/folders/3j/z5bj43t10vsdnks2s1xw3vm80000gn/T/ipykernel_94532/2823456958.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  re_red_chi2 = const_fn_red_chi2(re_dat, float(re_popt))
/var/folders/3j/z5bj43t10vsdnks2s1xw3vm80000gn/T/ipykernel_94532/2823456958.py:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this opera

In [802]:
##### FAKE ARRAY IN FORM OF KARTO'S DATA #####
fake_arr = np.tile([float(0)]*6, (20, 1))
for i in range(len(fake_arr)):
    fake_arr[i][1] = i
    fake_arr[i][2] = i
    real = 10
    imag = 0
    x = 1e-3
    re_noise = random.random() * x
    im_noise = random.random() * x
    if random.randint(0,1):
        re_noise *= -1
    if random.randint(0,1):
        im_noise *= -1
    fake_arr[i][3] = real + re_noise
    fake_arr[i][4] = imag + im_noise
    weight = (re_noise**2 + im_noise**2)**(-1)
    fake_arr[i][5] = weight
    fake_arr[i][3] *= weight # weighting reals like Karto
    fake_arr[i][4] *= weight # weighting imaginaries like Karto

In [803]:
mcmc_single_pt_source(fake_arr)

State([[10.00020127]
 [10.00017098]
 [10.00000421]
 [10.00010042]
 [ 9.99990837]
 [10.00013037]
 [10.00002711]
 [10.0000056 ]
 [ 9.99996078]
 [10.00005737]
 [10.0002355 ]
 [10.00004525]
 [ 9.99990367]
 [10.00002512]
 [10.00011746]
 [ 9.99998106]
 [10.00000585]
 [10.00003822]
 [10.00019585]
 [ 9.99995701]
 [10.00001636]
 [ 9.99981025]
 [ 9.99983941]
 [10.00016184]
 [ 9.99994238]
 [ 9.99994323]
 [ 9.99994498]
 [10.0000647 ]
 [10.00006722]
 [10.00003703]
 [10.00000797]
 [ 9.99999051]
 [ 9.99999271]
 [10.00002883]
 [10.0000283 ]
 [ 9.99994742]
 [ 9.99991401]
 [ 9.99992808]
 [10.00005705]
 [10.00001743]
 [10.00001565]
 [10.00004935]
 [10.00006557]
 [ 9.99996188]
 [10.00011412]
 [ 9.99998633]
 [ 9.99987669]
 [10.00011599]
 [10.00017431]
 [ 9.99995415]], log_prob=[242.19977524 242.88133773 244.32886544 243.96975926 243.39592972
 243.59298663 244.36110929 244.33291238 244.06574662 244.29093055
 241.27469556 244.33444839 243.317084   244.36122674 243.77078579
 244.2215269  244.33362421 244.3502

In [700]:
# ##### TESTING CHI2 METHOD ON FAKE ARRAY ######


# vis = np.array(fake_arr)
# u, v, Re, Im, w = [], [], [], [], []
# for row in vis:
#     u_data, v_data, Re_data, Im_data, w_data = row
#     u.append(int(u_data))
#     v.append(int(v_data))
#     Re.append(float(Re_data))
#     Im.append(float(Im_data))
#     w.append(float(w_data))

# # adding in conjugate half of data
# freq_bin *= 2
# neg_u = [-1 * val for val in u]
# u += neg_u
# neg_v = [-1 * val for val in v]
# v += neg_v
# Re *= 2
# neg_Im = [-1 * val for val in Im]
# Im += neg_Im
# w *= 2
# Rlambda = []
# for i in range(len(u)):
#     Rlambda.append((u[i]**2 +v[i]**2)**(1/2))

# sd = [weight**(-1/2) for weight in w]
# amp_dat = []
# re_dat = []
# im_dat = []
# for i in range(len(sd)):
#     amp_dat.append(((Re[i]**2 + Im[i]**2)**(1/2), sd[i]))
#     re_dat.append((Re[i], sd[i]))
#     im_dat.append((Im[i], sd[i]))
# zeroes = [0]*len(amp_dat)

# amp_popt, amp_pcov = curve_fit(const_fn_red_chi2, amp_dat, zeroes)
# amp_red_chi2 = const_fn_red_chi2(amp_dat, float(amp_popt))
# # amp_dict = {'amplitude': float(amp_popt[0]), 'standard deviation': float(amp_pcov[0][0]**(1/2)), 'reduced chi2': amp_red_chi2}
# print(amp_popt, amp_pcov[0][0]**(1/2), amp_red_chi2)

# re_popt, re_pcov = curve_fit(const_fn_red_chi2, re_dat, zeroes)
# re_red_chi2 = const_fn_red_chi2(re_dat, float(re_popt))
# print(re_popt, re_pcov[0][0]**(1/2), re_red_chi2)

# im_popt, im_pcov = curve_fit(const_fn_red_chi2, im_dat, zeroes)
# im_red_chi2 = const_fn_red_chi2(im_dat, float(im_popt))
# print(im_popt, im_pcov[0][0]**(1/2), im_red_chi2)

In [701]:
# ##### TESTING EMCEE METHOD ON FAKE ARRAY #####
# fake_arr = np.tile([float(0)]*5, (20, 1))

# import random
# for i in range(len(fake_arr)):
#     fake_arr[i][0] = i
#     fake_arr[i][1] = i
#     real = 10
#     imag = 0
#     x = 1e-3
#     re_noise = random.random() * x
#     im_noise = random.random() * x
#     if random.randint(0,1):
#         re_noise *= -1
#     if random.randint(0,1):
#         im_noise *= -1
#     fake_arr[i][2] = real + re_noise
#     fake_arr[i][3] = imag + im_noise
#     weight = (re_noise**2 + im_noise**2)**(-1)
#     fake_arr[i][4] = weight

# vis = np.array(fake_arr)
# amp, sd = [], []
# for row in vis:
#     u_data, v_data, Re_data, Im_data, w_data = row
#     amp.append(float((Re_data**2+Im_data**2)**(1/2)))
#     sd.append(float(w_data**(-1/2)))

# pos = np.random.randn(20,1) * 2 + 10
# nwalkers = 20
# ndim = 1
# sampler = emcee.EnsembleSampler(
#     nwalkers, ndim, log_probability, args=(np.array(amp), np.array(sd))
# )
# sampler.run_mcmc(pos, 5000)

In [702]:
# freq_bin, u, v, Re, Im, w = [], [], [], [], [], []
# for row in fake_arr:
#     u_data, v_data, Re_data, Im_data, w_data = row
#     freq_bin.append(int(freq_bin_data))
#     u.append(int(u_data))
#     v.append(int(v_data))
#     Re.append(float(Re_data))
#     Im.append(float(Im_data))
#     w.append(float(w_data))

In [703]:
# sd = [float(weight)**(-1/2) for weight in w]
# re_dat = []
# im_dat = []
# for i in range(len(sd)):
#     im_dat.append((Im[i], sd[i]))
#     re_dat.append((Re[i], sd[i]))
# zeroes = [0]*len(uv_dat)
# popt, pcov = curve_fit(red_chi2, im_dat, zeroes)
# red_chi2_im = red_chi2(im_dat, float(popt))
# popt2, pcov2 = curve_fit(red_chi2, re_dat, zeroes)
# red_chi2_re = red_chi2(re_dat, float(popt2))
# rchi2 = red_chi2_im + red_chi2_re

In [704]:
# pcov2

In [705]:
# popt2

In [706]:
# popt

In [707]:
# red_chi2_re

In [708]:
# rchi2

In [709]:
# Rlambda = []
# for i in range(len(u)):
#     Rlambda.append((u[i]**2 +v[i]**2)**(1/2))
# plt.scatter(Rlambda, Re)
# plt.ylim((0,12))

In [710]:
# plt.scatter(Rlambda, Im)
# plt.ylim((-1,1))